# Test IPYNB FILE

In [1]:
import sys
print(sys.executable)

c:\Users\dgalassi\commodity_lab\env_commodity_lab\Scripts\python.exe


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly as ply
import plotly.express as px
import seaborn as sns
import sys
import sqlite3 as sql

In [76]:
# connect to the database
conn = sql.connect("../../db/WTI_futures.db")
# load the table into a DataFrame
df = pd.read_sql('SELECT * FROM WTI_futures_data', conn)
conn.close()

In [77]:
#inspect
df.head()

,contract_id,close,change,open,high,low,previous,volume,open_interest,date
0,CLY00 (Cash),65.37,-1.84,0.00,65.37,65.37,67.21,0.0,0,2025-07-22 00:00:00
1,CLU25 (Sep '25),65.31,-0.64,65.77,65.86,64.82,65.95,258298.0,380321,2025-07-22 00:00:00
2,CLV25 (Oct '25),64.46,-0.56,64.84,64.95,64.00,65.02,135633.0,238748,2025-07-22 00:00:00
3,CLX25 (Nov '25),63.89,-0.53,64.25,64.35,63.47,64.42,81243.0,133633,2025-07-22 00:00:00
4,CLZ25 (Dec '25),63.49,-0.50,63.85,63.95,63.09,63.99,102265.0,243591,2025-07-22 00:00:00


In [78]:
#drop previous and change
df = df.drop(columns=['previous', 'change'])
#inspect
df.columns

Index(['contract_id', 'close', 'open', 'high', 'low', 'volume',
       'open_interest', 'date'],
      dtype='object')

In [79]:
#check missing entries
missing_entries = df.isnull().sum()
print(f"Missing entries per column: {missing_entries}")

Missing entries per column: contract_id       0
close             0
open              0
high              0
low               0
volume           18
open_interest     0
date              0
dtype: int64


In [80]:
df.contract_id.unique()

array(['CLY00 (Cash)', "CLU25 (Sep '25)", "CLV25 (Oct '25)",
       "CLX25 (Nov '25)", "CLZ25 (Dec '25)", "CLF26 (Jan '26)",
       "CLG26 (Feb '26)", "CLH26 (Mar '26)", "CLJ26 (Apr '26)",
       "CLK26 (May '26)", "CLM26 (Jun '26)", "CLN26 (Jul '26)",
       "CLQ26 (Aug '26)", "CLU26 (Sep '26)", "CLV26 (Oct '26)",
       "CLX26 (Nov '26)", "CLZ26 (Dec '26)", "CLF27 (Jan '27)",
       "CLG27 (Feb '27)", "CLH27 (Mar '27)", "CLJ27 (Apr '27)",
       "CLK27 (May '27)", "CLM27 (Jun '27)", "CLN27 (Jul '27)",
       "CLQ27 (Aug '27)", "CLU27 (Sep '27)", "CLV27 (Oct '27)",
       "CLX27 (Nov '27)", "CLZ27 (Dec '27)", "CLF28 (Jan '28)",
       "CLG28 (Feb '28)", "CLH28 (Mar '28)", "CLJ28 (Apr '28)",
       "CLK28 (May '28)", "CLM28 (Jun '28)", "CLN28 (Jul '28)",
       "CLQ28 (Aug '28)", "CLU28 (Sep '28)", "CLV28 (Oct '28)",
       "CLX28 (Nov '28)", "CLZ28 (Dec '28)", "CLF29 (Jan '29)",
       "CLG29 (Feb '29)", "CLH29 (Mar '29)", "CLJ29 (Apr '29)",
       "CLK29 (May '29)", "CLM29 (Jun '29)"

In [82]:
import pandas as pd
import numpy as np

def interpolate_missing_business_days(df):
    """
    Fill missing business days for each contract in the DataFrame.
    Interpolates 'open', 'high', 'low', 'close' linearly between previous and next business days.
    Orders contracts by numeric maturity extracted from contract_id.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain at least ['contract_id', 'date', 'open', 'high', 'low', 'close'].
    
    Returns
    -------
    pd.DataFrame
        DataFrame with missing business days filled per contract.
    """
        
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    #create a maturity order column based on the numeric part of contract_id (e.g., 'm003' -> 3)
    filled_all = []

    for contract_id in df['contract_id'].unique():
        contract_df = df[df['contract_id'] == contract_id].sort_values('date').reset_index(drop=True)
        
        #list of all business days between min and max date in the df -> missing dates will be the ones not in this list
        full_dates = pd.date_range(contract_df['date'].min(), contract_df['date'].max(), freq='B')
        missing_dates = full_dates.difference(contract_df['date'])
        
        rows_to_add = []

        for missing_date in missing_dates:
            #get previous and next date wrt to the missing date
            prev_rows = contract_df[contract_df['date'] < missing_date]
            next_rows = contract_df[contract_df['date'] > missing_date]

            if prev_rows.empty or next_rows.empty:
                continue
            
            prev_row = prev_rows.iloc[-1]
            next_row = next_rows.iloc[0]
            #interpolate linearly the values for the m# compute number of business days between prev and next
            #business-day-aware interpolation
            bdays_total = len(pd.bdate_range(prev_row['date'], next_row['date'])) - 1
            bdays_elapsed = len(pd.bdate_range(prev_row['date'], missing_date)) - 1
            interpolated_row = prev_row.copy()
            interpolated_row['date'] = missing_date
            for col in ['open', 'high', 'low', 'close']:
                interpolated_row[col] = prev_row[col] + (next_row[col] - prev_row[col]) * (bdays_elapsed / bdays_total)
            rows_to_add.append(interpolated_row)
        #concat to the original df
        if rows_to_add:
            contract_df = pd.concat([contract_df, pd.DataFrame(rows_to_add)], ignore_index=True)
            contract_df = contract_df.sort_values('date').reset_index(drop=True)

        filled_all.append(contract_df)

    filled_all = pd.concat(filled_all, ignore_index=True).reset_index(drop=True)
    return filled_all

test = interpolate_missing_business_days(df)

In [83]:
test

,contract_id,close,open,high,low,volume,open_interest,date
0,CLY00 (Cash),65.37,0.0,65.37,65.37,0.0,0,2025-07-22
1,CLY00 (Cash),65.24,0.0,65.24,65.24,0.0,0,2025-07-23
2,CLY00 (Cash),66.06,0.0,66.06,66.06,0.0,0,2025-07-24
3,CLY00 (Cash),65.17,0.0,65.17,65.17,0.0,0,2025-07-25
4,CLY00 (Cash),66.75,0.0,66.75,66.75,0.0,0,2025-07-28
...,...,...,...,...,...,...,...,...
6935,CLG36 (Feb '36),62.39,0.0,62.39,62.39,0.0,20,2025-09-30
6936,CLG36 (Feb '36),62.63,0.0,62.63,62.63,0.0,20,2025-10-01
6937,CLG36 (Feb '36),62.47,0.0,62.47,62.47,0.0,20,2025-10-02
6938,CLG36 (Feb '36),62.31,0.0,62.31,62.31,0.0,20,2025-10-03
